# Run this cell to load all required libraries 

In [1]:
%matplotlib qt
import visa
from visa import VisaIOError
from csv import reader
import numpy
import time, datetime

def parseCSVline(line):
    for lk in reader([line]):
        return lk
    
def avg(data):
    return sum(data)/len(data)

import time
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % (method.__name__, (te - ts) * 1000))
        return result
    return timed

class K194():
    '''This class adds automatic error checking to pyvisa write/query commands and also adds some common VXI/SCPI commands as class methods.'''
    def __init__(self, name, rm, address, visa_timeout_time=8000, visa_chunk_size=102400, debug=True):
        self.name = name
        self.debug = debug
        self.resource = rm.open_resource(address)
        #Timeout needs to be long enough for some of the slow devices to respond
        self.resource.timeout = visa_timeout_time
        #Chunk just needs to be big enough for any/all data transfers
        self.resource.chunk_size = visa_chunk_size
        
    def configure(self, samples, triggerV, rate, pretrigger = None):
        #C1X = Channel 1 and eXecute
        #R3 = 32V range
        #F0 = Waveform
        #X = eXecute
        #I0 = input DC coupled
        #S1,1E5 = Sample at 100,000Hz (1E5)
        #N0,32E3
        #111/10000=0.01 samples/s readback rate
        self.samples = samples
        self.rate = rate

        #CHANNEL 1
        self.resource.write("C01X")
        self.resource.write("R3F0I0S1,"+str(rate)+"N0,"+str(samples)+"X")
        self.pretrigger = 0
        if pretrigger != None:
            self.pretrigger = -int(samples * pretrigger)
            self.resource.write("W0,"+str(self.pretrigger)+"X")
        self.resource.write("T25X") #Trigger single acq from other channel
        
        #CHANNEL 2
        self.resource.write("C02X")
        self.resource.write("R3F0I0S1,"+str(rate)+"N0,"+str(samples)+"X")
        if pretrigger != None:
            self.resource.write("W0,"+str(self.pretrigger)+"X")
        if triggerV is None:
            self.resource.write("T7X") #Trigger on external line            
        else:
            self.resource.write("T21,"+str(triggerV)+"X") #Trigger single on +ve edge of triggerV.
            
        time.sleep(3) #These units take ages to complete configuration

        
    def triggerNow(self):
        self.resource.write("C01X")
        self.resource.write("T27X") #Trigger immediately.
        
    def read(self):
        old_timeout = self.resource.timeout
        self.resource.timeout = None
        print("Reading K194 channel 1, started at",datetime.datetime.now().time(), "finish at", (datetime.datetime.now()+datetime.timedelta(seconds=self.samples*0.0112)).time())
        self.resource.write("G4X")
        self.resource.write("C01X")
        self.resource.write("B0X") #Reset to start of buffer (and channel 1)
        start = time.perf_counter()
        ch1 = self.resource.read()
        #print(time.perf_counter(), start, self.samples, time.perf_counter() - start, self.samples*0.0112)
        print("Reading rate", (time.perf_counter() - start) / self.samples, "samples/s")
        print("Reading K194 channel 2, started at",datetime.datetime.now().time(), "finish at", (datetime.datetime.now()+datetime.timedelta(seconds=self.samples*0.0112)).time())
        self.resource.write("C02X")
        self.resource.write("B0X") #Reset to start of buffer for channel 2
        start = time.perf_counter()
        ch2 = self.resource.read()
        #print(time.perf_counter(), start, self.samples, time.perf_counter() - start)
        print("Reading rate", (time.perf_counter() - start) / self.samples, "samples/s")
        print("Read complete at",datetime.datetime.now().time())
        self.resource.timeout = old_timeout
        return list(map(float, ch1.split(','))), list(map(float, ch2.split(',')))

    def status(self):
        status_word = self.resource.query("C01XU2X")
        return {
            "Buffer full":status_word[3] == "1",
            "Buffer half full":status_word[4] == "1",
            "Plotter done":status_word[5] == "1",
            "CH1 armed":status_word[6] == "1",
            "CH2 armed":status_word[7] == "1",
            "16 bit":status_word[8] == "1",
        }
    
    def wait_till_triggered(self):
        while(self.status()["CH1 armed"]):
            print("Waiting for K194 Ch1 trigger complete")
            time.sleep(1)

class Instrument():
    '''This class adds automatic error checking to pyvisa write/query commands and also adds some common VXI/SCPI commands as class methods.'''
    def __init__(self, name, rm, address, full_test, visa_timeout_time=8000, visa_chunk_size=102400, no_check_commands=[], debug=True):
        self.name = name
        self.full_test = full_test
        self.debug = debug
        self.no_check_commands = no_check_commands #Commands which lock-up or reset the instrument, so checking for errors doesn't work
        self.resource = rm.open_resource(address)
        #Timeout needs to be long enough for some of the slow devices to respond
        self.resource.timeout = visa_timeout_time
        #Chunk just needs to be big enough for any/all data transfers
        self.resource.chunk_size = visa_chunk_size
        
    def query(self, cmd):
        '''Send a SCPI query and test/capture any instrument errors'''
        retval = None
        try:
            retval = self.resource.query(cmd)
        except Exception as e:
            #If the command fails/timeouts, check for errors
            raise Exception("Instrument Errors: "+repr(self.get_errors())+"\nLeading to "+str(e))
        #If it succeeds, still check for errors
        if cmd not in self.no_check_commands and self.debug:
            self.check_errors(cmd)
        return retval
    
    def write(self, cmd):
        '''Send a SCPI write and test/capture any instrument errors'''
        try:
            self.resource.write(cmd)
        except Exception as e:
            #If the command fails/timeouts, check for errors
            raise Exception("Instrument Errors: "+repr(self.get_errors())+"\nLeading to "+str(e))
        #If it succeeds, still check for errors
        if cmd not in self.no_check_commands and self.debug:
            self.check_errors(cmd)
    
    def get_errors(self):
        '''Get a list of errors of the device, if there's no error return an empty list'''
        errors=[]
        while True:
            try:
                errcode, errstring = parseCSVline(self.resource.query("SYSTEM:ERROR?"))
            except VisaIOError as e:
                #If the command fails/timeouts, check for errors
                raise Exception("VisaIOError on "+self.name+"\nLeading to "+str(e))
            errcode = int(errcode)
            if errcode == 0:
                #This isn't an error, all is fine so stop parsing here
                break
            errors.append([errcode, errstring])
        #Finally, we clear the instrument status if its errored
        if len(errors) != 0:
            self.resource.write("*CLS")
        return errors
    
    def check_errors(self, cmd=None):
        '''Checks if the instrument has any errors and, if so, raises an exception'''
        errors = self.get_errors()
        if len(errors) != 0:
            if cmd != None:
                raise Exception(self.name+": cmd:"+repr(cmd)+" gives errors "+repr(errors))
            else:
                raise Exception(self.name+": Errors "+repr(errors))
            
    ######## SCPI command helpers
    def reset(self):
        self.write("*RST")

class DMMX5XX(Instrument):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def fetchTrace(self, binary=True):
        self.write("*WAI")#Ensure trace is ready
        count = self.query(':TRACE:ACTUAL?').strip()
        
        if binary:
            self.write("FORM REAL")
            DMMdata = self.resource.query_binary_values('TRAC:DATA? 1, '+count+', "defbuffer1", READ, REL', is_big_endian=False, datatype=u'd')
            self.write("FORM ASCii")
        else:
            DMMdata = self.resource.query_ascii_values('TRAC:DATA? 1, '+count+', "defbuffer1", READ, REL')
        
        start = int(self.query(":TRACE:ACTUAL:START?"))
        end = int(self.query(":TRACE:ACTUAL:END?"))
        if end > start:
            sliceend = end
        else:
            sliceend = -1
        slice_data = DMMdata[2*start:2*sliceend]
        if sliceend == -1:
            slice_data = slice_data + DMMdata[:2*end]
        return [x-slice_data[1] for x in slice_data[1::2]], slice_data[0::2]
    
    def readData(self, t="VOLT:DC", channels = None):
        cmds = ["*RST",
        'SENS:FUNC "'+t+'", (@'+channels+')',
        ":SENS:VOLT:RANGE:AUTO ON",
        ":SENS:VOLT:INP AUTO", 
        ":SENS:VOLT:NPLC 5",
        ":SENS:VOLT:AZER ON",
        ':ROUTe:OPEN:ALL',
               ]
        
        if channels != None:
            cmds += [':ROUTe:CLOSE (@'+channels+')']
        
        for cmd in cmds:
            self.write(cmd)
        return float(self.query("READ?"))

    def readV(self, channels = None):
        return self.readData("VOLT:DC", channels)

    def readFR(self, channels = None):
        return self.readData("FRESistance", channels) 

    def readR(self, channels = None):
        return self.readData("RESistance", channels) 
    
    def readI(self, channels = None):
        return self.readData("CURR:DC", channels)
    
class Agilent66311B(Instrument):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def writeDisplay(self, val):
        PSU.write("DISPLAY:MODE TEXT")
        PSU.write('DISPLAY:TEXT "'+str(val)[:14]+'"')
        
    def output(self, Vmax = None, Imax = None, enabled=None):
        if Vmax != None:
            self.write("VOLT "+str(Vmax))
        if Imax != None:
            self.write("CURR "+str(Imax)) #Allow the max current
        if Imax != None or Vmax != None:
            self.write("*WAI")
        if enabled == True:
            self.write("OUTP ON")
        elif enabled == False:
            self.write("OUTP OFF")

    def setoutputAtTrigger(self, voltage):
        #Setup the voltage for the trigger system 1 (output)
        self.write("VOLT:TRIG "+str(voltage))
        #Initialise it ready for the trigger
        self.write("INIT:SEQ1")
        #That's it, the only trigger source for output is the BUS
        #Setup the measurement
        self.write("TRIG:ACQ:SOUR BUS") #Trigger when the output is triggered
            
    def resetDisplay(self):
        self.write("DISPLAY:MODE NORM")
    
    def readIArray(self, *args, **kwargs):
        return self.readDataArray("CURR", *args, **kwargs)
    
    def readVArray(self, *args, **kwargs):
        return self.readDataArray("VOLT", *args, **kwargs)
    
    def readDataArray(self, t="CURR", binary=True, count=2048, fetch_old=False):
        if not 0 < count <= 2049:
            raise RuntimeException("Bad data count of "+str(count))

        if fetch_old:
            get = "FETCH"
        else:
            get = "MEAS"
            self.write("SENS:SWE:POIN "+str(count))
        
        if binary:
            self.write("FORM REAL")
            val = self.resource.query_binary_values(get+":ARRAY:"+t+"?", is_big_endian=True)
            self.write("FORM ASCii")
            return val
        else:
            return self.resource.query_ascii_values(get+":ARRAY:"+t+"?")
        
    def readI(self, *args, **kwargs):
        return self.readData("CURR", *args, **kwargs)
    
    def readV(self, *args, **kwargs):
        return self.readData("VOLT", *args, **kwargs)
    
    def readData(self, t="CURR"):
        return list(map(float, [self.query("MEAS:"+t+"?"), self.query("FETC:"+t+":MAX?"), self.query("FETC:"+t+":MIN?")]))
    
    
    
    def conf_measure(self, triggerLevel = None, triggerType="VOLT", triggerSlope=1, triggerHyst=None,
                     measType="CURR", points = 4096, pointsOffset = 0, dt = 15.6e-6):
        #Trigger type can be VOLT or CURR or BUS, lets get it set up
        if (triggerType == "VOLT" or triggerType == "CURR") and isinstance(triggerLevel, float):
            self.write("TRIG:ACQ:SOUR INT")
            self.write("TRIG:ACQ:LEV:"+triggerType+" "+str(triggerLevel))
            
            if isinstance(triggerHyst, float):
                self.write("TRIG:ACQ:HYST:"+triggerType+" "+str(triggerHyst))#prevent spurious changes
            elif triggerHyst != None:
                raise RuntimeError("Bad triggerHyst type, "+str(triggerHyst))
                
            if triggerSlope == 1:
                self.write("TRIG:ACQ:SLOP:"+triggerType+" POS")
            elif triggerSlope == 0:
                self.write("TRIG:ACQ:SLOP:"+triggerType+" EITH")
            elif triggerSlope == -1:
                self.write("TRIG:ACQ:SLOP:"+triggerType+" NEG")
            else:
                raise RuntimeError("Invalid trigger slope"+str(triggerSlope))
        elif triggerType == "BUS":
            self.write("TRIG:ACQ:SOUR BUS")
        else:
            raise RuntimeError("Invalid trigger type, "+str(triggerType))
    
        if measType == "VOLT" or measType == "CURR":  
            PSU.write('SENS:FUNC "'+measType+'"')
        else:
            raise RuntimeError("Bad measType")
    
        if not 1 <= points <= 4096:
            raise RuntimeError("Bad points, "+str(points))
            
        self.write('SENS:SWE:POIN 4096') #Max 4096 samples allowed
        self.write('SENS:SWE:TINT '+str(dt)) #Set sample time
        #readback dt as the PSU rounds to the nearest 15.6e-6 s
        self.dt = float(PSU.query("SENSe:SWEep:TINTerval?"))

        if not -4095 <= pointsOffset <= 2e9:
            raise RuntimeError("Bad pointsOffset, "+str(pointsOffset))
        self.write("SENS:SWE:OFFS:POIN "+str(pointsOffset)) #from -4095 to 2e9 points offset from trigger
        self.write("INIT:NAME ACQ")

    def busTrigger(self):
        self.write("*TRG")
        self.write("*WAI")

rm = visa.ResourceManager()
DMMV = DMMX5XX('DMM6500', rm, 'TCPIP0::10.0.0.12::inst0::INSTR', full_test=False)
#DMMV = DMMX5XX('DMM7510', rm,  'TCPIP0::10.0.0.8::inst0::INSTR', full_test=False)
PSU = Agilent66311B('66311B', rm, 'GPIB0::5::INSTR', full_test=False)        
k194 = K194("K194", rm, 'GPIB0::17::INSTR')
print("DONE")

DONE


In [ ]:
print(DMMV.query("*IDN?"),PSU.query("*IDN?"))

# New Acquisition script

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import math

#Our probe is this
#https://uk.rs-online.com/web/p/platinum-resistance-temperature-sensors/2364299/?relevancy-data=636F3D3126696E3D4931384E525353746F636B4E756D626572266C753D656E266D6D3D6D61746368616C6C26706D3D5E2828282872737C5253295B205D3F293F285C647B337D5B5C2D5C735D3F5C647B332C347D5B705061415D3F29297C283235285C647B387D7C5C647B317D5C2D5C647B377D2929292426706F3D3126736E3D592673723D2673743D52535F53544F434B5F4E554D4245522677633D4E4F4E45267573743D32333634323939267374613D3233363432393926&searchHistory=%7B%22enabled%22%3Atrue%7D
#1/5th DIN PT100 +-0.06C at 0C (in accordance with IEC 751)
#This means the alpha is 0.00385 
#Following the notes at http://educypedia.karadimov.info/library/c15_136.pdf
#We can use the Callendar-Van Dusen expression to calculate T
#
# Here's a definition via the ITS-90 versus the IPTS-68
#http://www.code10.info/index.php%3Foption%3Dcom_content%26view%3Darticle%26id%3D82:measuring-temperature-platinum-resistance-thermometers%26catid%3D60:temperature%26Itemid%3D83
def RTD_RtoT(R, R0=100):
    A = 3.9083E-3
    B = -5.775E-7
    #C = -4.183E-12
    Temp = (-R0 * A + math.sqrt(R0**2 * A**2 - 4 * R0 * B * (R0 - R)))  / (2 * R0 * B)
    return Temp

#Our wire lengths are 89.50mm and 60.50mm from pad end to pad end, and 0.015mm diameter
#Short and Long resistances per length should be within 2% (pg 463 NIST THW)
#Calculations for platinum with a resistivity of 10.6e-8 Ohm m, and diameter 0.015mm
#Gives 53.685438 Ohm for 8.95cm and 36.Ohm for 6.05cm        
wire_length_long = 0.0895 #m
wire_length_short = 0.0605 #m
# Measured in-place wire resistances are 56.3028 and 38.1656 with contact/lead resistance of 0.006, but much more variance in readings than that
def Long_HW_RtoT(R):
    Rpt =  1.9192360733223648 * R + 0.6772834578915337
    return RTD_RtoT(Rpt)

def Short_HW_RtoT(R):
    Rpt =  2.811359007746365 * R + 1.6831528533284796
    return RTD_RtoT(Rpt)

Rsense=98.96

def ReadPT100R(long=False):
    DMMV.write("*RST")
    DMMV.write('SENS:FUNC "FRES", (@1)')
    DMMV.write('FRES:NPLC 5')
    DMMV.write('FRES:RANGE:AUTO ON')
    DMMV.write('FRES:OCOM ON')
    if long:
        DMMV.write("ROUT:CLOS (@1, 6)")
    if short:
        DMMV.write("ROUT:CLOS (@2, 7)")
    return float(DMMV.query("READ?"))

    PSU.output(Vmax=0.1, Imax=3.0, enabled=True)
    time.sleeop(1) # allow supply to stabilise

def ReadDMMVolt(chan):
    DMMV.write("*RST")
    DMMV.write('SENS:FUNC "VOLT", (@'+str(chan)+')')
    DMMV.write('VOLT:RANGE:AUTO ON')
    DMMV.write("VOLT:AZER ON")
    DMMV.write("ROUT:CLOS (@"+str(chan)+")")
    return float(DMMV.query("READ?"))

def ReadTemperatures(V = 0.01):
    PSU.output(Vmax=V, Imax=3.0, enabled=True)
    time.sleep(1)
    PT100_long = ReadPT100R(long=True)
    PT100_short = ReadPT100R(long=False)
    VIsense = ReadDMMVolt(2)
    I = VIsense / Rsense 
    LWire = ReadDMMVolt(3)
    SWire = ReadDMMVolt(4)
    return [PT100_long, PT100_short, LWire / I, SWire / I]
    
@timeit
def runTest(current, pretrigger=0.05):
    Rlong = 60.0 #Estimate of the long wire resistance, so we can calculate ranges etc
    
    DMMChannel = "SW" # "LW" = Long Wire, "SW" = short wire, "I" = current sense resistor
    run_time = 2 # Seconds to sample for

    #More samples means more data to transfer (slower), but also, more data
    #Higher sample rates mean better resolution, but less time measured. 
    if True: #Do it fast for testing
        DMMsamples = 2000
        PSUsamples = 4096 # Max is 4096
        k194samples = 100 # Max is 32768 samples
    else:
        DMMsamples = 200000 
        PSUsamples = 4096 # Max is 4096
        k194samples = 1000 # Max is 32768 samples
    
    #Calculate sample rate from run_time
    DMMsamplerate = DMMsamples / run_time
    PSUsamplerate = min(64103, PSUsamples / run_time) # Max is about 64103 Hz (1/15.6e-6), actual is calculated by the PSU
    k194samplerate = min(100000, k194samples / run_time) # Max is 100k
    
    #dt above is the shortest value possible
    PSU.writeDisplay(str(current*1000)+"mA Init")
    #voltage=15
    #current = voltage / 200.0
    voltage = current * 200.0
    
    #Start the power supply up, get it holding the voltage at zero volts
    PSU.output(Vmax=0.0, Imax=3.0, enabled=True)
    #Set the PSU target voltage (once triggered)
    PSU.setoutputAtTrigger(voltage=voltage)
    
    #Wait for it to settle before setting up triggers and measure offset voltages
    time.sleep(1)
    
    #Read offset voltages
    #IVoffset = DMMI.readV()
    
    PSUoffset = -int(pretrigger * PSUsamples)
    #PSU.conf_measure(triggerLevel=0.1, triggerType="VOLT", triggerSlope=1, triggerHyst=0.01, 
    #                 measType="CURR", points=PSUsamples, pointsOffset = PSUoffset, dt = 1.0/PSUsamplerate)
    PSU.conf_measure(triggerType="BUS", measType="CURR", points=PSUsamples, pointsOffset = PSUoffset, dt = 1.0/PSUsamplerate)
    #if current < 0.020:
    #    PSU.write("SENS:CURR:RANG MIN")
    #else:
    #    PSU.write("SENS:CURR:RANG MAX")
    PSU.write("SENS:CURR:RANG MAX")
    #PSU.write("SENS:CURR:DET DC") # For pure DC waveforms (not us! causes massive overshoot)
    PSU.write("SENS:CURR:DET ACDC") # for potential offsets of up to 2mA, but fast response
    PSU_srate = int(1.0/15.6e-6) #Sample rate per second (1/15.6e-6=64102.5641), max is 10^6
    
    cmds = ["*RST",
    ':SENS:DIG:FUNC "VOLT"',
    ':DIG:VOLT:RANGE 10',
    ":SENS:DIG:VOLT:SRATE "+str(DMMsamplerate),
    ':TRACe:POINts '+str(DMMsamples)+', "defbuffer1"',
    ### Externally triggered measurements
    #":TRIG:EXT:IN:EDGE FALL",
    #':TRIGger:LOAD "LoopUntilEvent", EXTernal, '+str(pretrigger*100)+', ENTer, 0, "defbuffer1"', #Loop until event keeping up to 2% of pre-trigger data
    ### Internally triggered measurements
    ":DIG:VOLT:ATR:EDGE:SLOP RIS",
    ":DIG:VOLT:ATR:MODE EDGE",
    ":DIG:VOLT:ATR:EDGE:LEV "+str(0.1*current*Rlong), # 1% of the predicted long wire V drop
    ':TRIGger:LOAD "EMPTY"',
    ':TRIGger:BLOCk:BUFFer:CLEAr 1, "defbuffer1"',
    ':TRIGger:BLOCk:MDIGitize 2, "defbuffer1", INF',
    ':TRIGger:BLOCk:WAIT 3, ATRigger',
    ':TRIGger:BLOCk:NOTify 4, 1',
    ':TRIGger:BLOCk:MDIGitize 5, "defbuffer1", '+str(int(DMMsamples*0.98)),
    ':TRIGger:EXTernal:OUT:STIMulus NOTify1'
    ]
    
    if DMMChannel == "LW":
        cmds += ['ROUT:CLOS (@3)'] #Channel 3 is the Long wire, 2 is the current sense, 1 (and 8) is the RTD.
    elif DMMChannel == "SW":
        cmds += ['ROUT:CLOS (@4)'] #Channel 2 is the current sense
    elif DMMChannel == "I":
        cmds += ['ROUT:CLOS (@8)'] #Channel 2 is the current sense
    else:
        raise RuntimeError("Invalid DMM channel")
    
    cmds += [
    ':DISPlay:SCReen GRAPh',
    '*WAI',
    ':INIT']

    for cmd in cmds:
        DMMV.write(cmd)
        
    PSU.writeDisplay("Sleeping for DMM6500 1")
    time.sleep(2)
    #k194.configure(samples = k194samples, triggerV = 0.01, rate=k194samplerate, pretrigger=pretrigger)
    k194.configure(samples = k194samples, triggerV = None, rate=k194samplerate, pretrigger=pretrigger)

    #Ready the trigger system for the run
    PSU.writeDisplay(str(current*1000)+"mA RUN")
    #Trigger the run
    PSU.busTrigger()
    
    k194.wait_till_triggered()
        
    PSU.writeDisplay(str(current*1000)+"mA SLEEP")
    time.sleep(DMMsamples / DMMsamplerate)
    
    #Disable the PSU
    PSU.output(Vmax=0, enabled=False)
    
    PSU.writeDisplay("D/L DMM V")
    t_V, V = DMMV.fetchTrace()
    VDMM = pd.Series(V, index=t_V)
    
    PSU.writeDisplay("D/L PSU I")
    Ipsu = PSU.readIArray(fetch_old=True)
    t_Ipsu = [(i - PSUoffset) * PSU.dt for i in range(len(Ipsu))]
    Ipsu = pd.Series(Ipsu, index=t_Ipsu)

    PSU.writeDisplay("D/L K194")
    ch1, ch2 = k194.read()
    t_k194 = [i / k194.rate for i in range(len(ch1))]
    K194V1 = pd.Series(ch1, index=t_k194)
    K194V2 = pd.Series(ch2, index=t_k194)
    
    df = pd.DataFrame({'VDMM':VDMM, 'Ipsu':Ipsu, "K194V1": K194V1, "K194V2":-K194V2})
    df.index.names = ['t']
    return df

run = True
plot = True
current=0.05 #0.075 is the max

if run:
    data = runTest(current=current)
    PSU.writeDisplay(str(current*1000)+"mA Proc")    

#print(data.VDMM.max())
    
if plot:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title("Raw measure data")
    data.VDMM.plot(ax=ax, style="1", markerfacecolor="none", markeredgecolor='#FF0000')
    data.K194V1.plot(ax=ax, style="2", markerfacecolor="none", markeredgecolor='#FF8080')
    data.K194V2.plot(ax=ax, style="3", markerfacecolor="none", markeredgecolor='#FF4040')
    ax.set_ylabel("$V$ (V)", color="tab:red")
    ax.tick_params(axis='y', labelcolor="tab:red")
    
    ax2 = data.Ipsu.plot(secondary_y=True, ax=ax, style=".b")
    #data.Ipsu.plot(secondary_y=True, ax=ax, style=":", color="tab:blue")
    ax2.set_ylabel("$I$ (A)", color="tab:blue")
    ax2.tick_params(axis='y', labelcolor="tab:blue")
    fig.legend()

if run:
    PSU.writeDisplay("Plot interp data")
    data = data.interpolate(method='linear')

if plot:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title("Interpolated measure data")
    data.V.plot(ax=ax, style="-", color="tab:red")
    ax2 = data.I.plot(secondary_y=True, ax=ax, style="")
    data.Ipsu.plot(secondary_y=True, ax=ax, style="--", color="tab:blue")
    ax.set_ylabel("$V$ (V)", color="tab:red")
    ax2.set_ylabel("$I$ (A)", color="tab:blue")
    ax.tick_params(axis='y', labelcolor="tab:red")
    ax2.tick_params(axis='y', labelcolor="tab:blue")
    fig.legend()

if run:
    #Trim off the zero time results
    PSU.writeDisplay("Calc R and Q")
    data = data[data.index > 0]
    data['R'] = data.V / data.I
    data['q'] = data.V * data.I / wire_length_long

if plot:
    PSU.writeDisplay("Plot R, Q")
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title("Resistance")
    data['R'].plot(ax=ax, color="tab:red")
    ax2 = data['q'].plot(secondary_y=True, ax=ax, color="tab:blue")
    ax.set_ylabel("$R$ (Ohm)", color="tab:red")
    ax2.set_ylabel("$Q$ (W)", color="tab:blue")
    ax.set_xlabel("t (s)")
    ax.tick_params(axis='y', labelcolor="tab:red")
    ax2.tick_params(axis='y', labelcolor="tab:blue")

    PSU.writeDisplay("Plot T/DT/ln t")

if run:
    PSU.writeDisplay("Calc T and ln t")
    data['T'] = data.R.apply(Long_HW_RtoT)
    data.reset_index(level=0, inplace=True)
    data['ln t'] = data['t'].transform(lambda t : numpy.log(t*1000))

if plot:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    data.plot('t', 'T', ax=ax)


if run:
    PSU.writeDisplay("Writing excel")
    data.to_csv('I'+str(current*1000)+'mA.csv')

PSU.writeDisplay("Complete!")

Reading K194 channel 1, started at 08:33:53.462112 finish at 08:33:54.582112
Reading rate 0.011276730000000157 samples/s
Reading K194 channel 2, started at 08:33:54.925255 finish at 08:33:56.045255
Reading rate 0.011374297000000127 samples/s
Read complete at 08:33:56.287473
'runTest'  19216.50 ms


AttributeError: 'DataFrame' object has no attribute 'V'

In [ ]:
k194.resource.write("C02X")
k194.resource.write("B0X")
k194.resource.read()

In [ ]:
DMMV.readFR(channels="1, 6") # RTD channels

In [ ]:
data["K194V2"].dropna()


In [ ]:
PSU.output(Vmax=0.1, Imax=3.0, enabled=True)

In [ ]:
#Measured in-place wire resistances are 56.3028 and 38.1656
print(RTD_RtoT(107.7626))
for i in range(30):
    print(ReadTemperatures(V=0.01))
    time.sleep(1)

In [ ]:
print(RTD_RtoT(108.7594))
